In [1]:
#pip install unidecode

In [2]:
from bs4 import BeautifulSoup as bs
import requests
from pprint import pprint
import re
import unidecode
import pandas as pd
from pymongo import MongoClient
from pprint import pprint

Здесь надо ввести текст для поиска вакансии

In [3]:
text= 'python'

In [4]:
main_link = 'https://hh.ru'
params = {'L_save_area':'false',
          'clusters':'true',
         'enable_snippets':'true',
         'showClusters':'true'}
page = '0'


In [5]:
params['text'] = text
params['page'] = page

In [6]:
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.72 Safari/537.36'}

In [7]:
response = requests.get(main_link+'/search/vacancy', params=params, headers=headers)

In [8]:
def get_salary_from_string(salary_str):
    salary_str = vacancy_comp.getText()
    a = salary_str.split(' ')
    currency = a[-1]
    #смотрим - указан ли в зп диапазон
    salary_number = a[-2].split('-')
    #если да - то обрабатываем ситуацию, когда есть и максимальная, и минимальная зп
    if (len(salary_number)==2):
        try:
            salary_min = \
            int(unidecode.unidecode(salary_number[0]).replace(' ',''))
        except Exception as e:
            salary_min = None
        try:   
            salary_max = \
            int(unidecode.unidecode(salary_number[1]).replace(' ',''))
        except Exception as e:
            salary_max = None
    #если нет - то обрабатываем ситуацию, когда есть только минимальная зп
    else: 
         if a[0]=='от':
            salary_max=None
            try:
                salary_min = \
                int(unidecode.unidecode(salary_number[0]).replace(' ',''))
            except Exception as e:
                salary_min = None
                        
         #или когда есть только максимальная зп
         else:
                if a[0]=='до':   
                    salary_min=None
                    try:
                        salary_max = \
                        int(unidecode.unidecode(salary_number[0]).replace(' ',''))
                    except Exception as e:
                        salary_max = None

    return salary_max, salary_min, currency           


In [9]:
vacancies=[]
while True:
    next_page=None
    if response.ok:
        soup = bs(response.text, 'html.parser')
        next_page = soup.find('a',{'data-qa':"pager-next"})
        vacancies_list = soup.findAll('div', {'class':'vacancy-serp-item'})
        
        
        for vacancy in vacancies_list:
            vacancy_data = {}
            vacancy_all = vacancy.find('a',{'data-qa':'vacancy-serp__vacancy-title'})
            vacancy_link = vacancy_all['href']
            vacancy_name = vacancy_all.getText()
            vacancy_site = "hh.ru"
            vacancy_comp = vacancy.find('span',{'data-qa':'vacancy-serp__vacancy-compensation'})
            if (vacancy_comp!=None):
                #обрабатываем строку с зарплатой
                salary_max, salary_min, currency = get_salary_from_string(vacancy_comp.getText())
            else: 
                salary_max=None
                salary_min=None
                currency= None
            vacancy_data['name'] = vacancy_name
            vacancy_data['link'] = vacancy_link
            vacancy_data['from_site'] = vacancy_site
            vacancy_data['min_compensation'] = salary_min
            vacancy_data['max_compensation'] = salary_max
            vacancy_data['currency'] = currency
            vacancies.append(vacancy_data)
            
        if (next_page!=None):
            next_page_link = main_link+next_page['href']
            response = requests.get(next_page_link, headers=headers)
        else: break

In [10]:
df = pd.DataFrame(vacancies)

In [11]:
df

,name,link,from_site,min_compensation,max_compensation,currency
0,Python Developer,https://samara.hh.ru/vacancy/43048662?query=py...,hh.ru,NaN,NaN,None
1,Python Developer,https://samara.hh.ru/vacancy/43046900?query=py...,hh.ru,NaN,NaN,None
2,Data Analyst,https://samara.hh.ru/vacancy/43075693?query=py...,hh.ru,NaN,NaN,None
3,Quantitative developer (HFT developer),https://samara.hh.ru/vacancy/43005027?query=py...,hh.ru,NaN,400000.0,руб.
4,Quantitative developer (HFT developer),https://samara.hh.ru/vacancy/43005025?query=py...,hh.ru,NaN,400000.0,руб.
...,...,...,...,...,...,...
1995,Инженер по автоматическому тестированию (Python),https://samara.hh.ru/vacancy/42663888?query=py...,hh.ru,NaN,NaN,None
1996,Backend Developer,https://samara.hh.ru/vacancy/42228177?query=py...,hh.ru,140000.0,200000.0,руб.
1997,Python developer Банк 131,https://samara.hh.ru/vacancy/42629229?query=py...,hh.ru,NaN,NaN,None
1998,Программист Python,https://samara.hh.ru/vacancy/42721797?query=py...,hh.ru,90000.0,145000.0,руб.


Урок 3
1. Развернуть у себя на компьютере/виртуальной машине/хостинге MongoDB и реализовать функцию, записывающую собранные вакансии в созданную БД.

In [12]:
client = MongoClient('localhost',27017)
db = client['vacancies1503']
vac_col = db.vacancies

In [13]:
def safe_vacancies(list_vacancies, collection ):
    collection.insert_many(list_vacancies)


In [14]:
safe_vacancies(vacancies, vac_col)

In [15]:
#функция выводит в список результат запроса к mongodb
def show_result(result1):
    result1_list=[]
    for item in result1:
        result1_list.append(item)
    return result1_list

2. Написать функцию, которая производит поиск и выводит на экран вакансии с заработной платой больше введённой суммы.

In [26]:
def find_vacancy(min_salary, collection):
    return collection.find({'$or':[{'min_compensation':{'$gt' : min_salary}},{'max_compensation':{'$gt' : min_salary}}]})
    

In [27]:
#так как не знаю, как в Юпитере реализовать интеррактивный 
#ввод параметров для поиска, просто жестко пишу его при вызове
result_list = show_result(find_vacancy(140000, vac_col))


In [28]:
df_result = pd.DataFrame(result_list)
df_result

,_id,name,link,from_site,min_compensation,max_compensation,currency
0,60510a0b98a9163522574faf,Quantitative developer (HFT developer),https://samara.hh.ru/vacancy/43005027?query=py...,hh.ru,NaN,400000.0,руб.
1,60510a0b98a9163522574fb0,Quantitative developer (HFT developer),https://samara.hh.ru/vacancy/43005025?query=py...,hh.ru,NaN,400000.0,руб.
2,60510a0b98a9163522574fb4,DevOps Engineer,https://samara.hh.ru/vacancy/42301915?query=py...,hh.ru,170000.0,250000.0,руб.
3,60510a0b98a9163522574fbc,Python-разработчик,https://samara.hh.ru/vacancy/42277259?query=py...,hh.ru,160000.0,200000.0,руб.
4,60510a0b98a9163522574fbd,Программист Python,https://samara.hh.ru/vacancy/42921655?query=py...,hh.ru,150000.0,200000.0,руб.
...,...,...,...,...,...,...,...
1543,6051a3cf515912e0680f4d29,Python-developer,https://samara.hh.ru/vacancy/42571969?query=py...,hh.ru,95000.0,150000.0,руб.
1544,6051a3cf515912e0680f4d50,Devops-инженер,https://samara.hh.ru/vacancy/42840328?query=py...,hh.ru,150000.0,200000.0,руб.
1545,6051a3cf515912e0680f4d54,Deep Learning Engineer,https://samara.hh.ru/vacancy/41525608?query=py...,hh.ru,100000.0,200000.0,руб.
1546,6051a3cf515912e0680f4d63,Backend Developer,https://samara.hh.ru/vacancy/42228177?query=py...,hh.ru,140000.0,200000.0,руб.


3. Написать функцию, которая будет добавлять в вашу базу данных только новые вакансии с сайта.

In [19]:
#выбрала в качестве индентификатора уникальности вакансии ссылку на нее

In [20]:
def add_new_vacancies(item, collection):
    result_list =[]
    result_func = collection.find({'link' : item['link']})
    for item in result_func:
        result_list.append(item)
    if (len(result_list)==0):
        collection.insert_one(item)

In [21]:
new_item = {'name': 'Программист - биостатистик',
  'link': 'https://samara.hh.ru/vacancy/43089610?query=python88',
  'from_site': 'hh.ru',
  'min_compensation': None,
  'max_compensation': None,
  'currency': None}

In [22]:
add_new_vacancies(new_item, vac_col)

In [23]:
result1_list = show_result(vac_col.find({'link' : new_item['link']}))


In [24]:
result1_list

[{'_id': ObjectId('60519f1b06f379d5a5196522'),
  'name': 'Программист - биостатистик',
  'link': 'https://samara.hh.ru/vacancy/43089610?query=python88',
  'from_site': 'hh.ru',
  'min_compensation': None,
  'max_compensation': None,
  'currency': None}]